In [ ]:
!pip install gymnasium
!pip install plotly
from myst_nb import glue

# Let's practice

We will see a very simle grid world problem. In this environment the agent start always in the same position and needs to reach a cell in the world.
To complicate things for the agent, one cell is a trap and if the agent go on it the game is lost.

Visualy the world will look like this:

```{figure} /lectures/mdp/dynamic-programming/grid_world.png
:align: center
:width: 70%
```

## Markov Decision Process

Before implementing the environment and solving it, we need to define it as a MDP $\Omega = (S,A,T,R)$, where:
- $S$ is the state space: each state $s$ will be a cell of the grid such as $\forall s \in S, s \in [0, 11]$.
- $A$ is the action space: each action $a$ will be a direction such as $\forall a \in A, a \in \{UP, DOWN, LEFT, RIGHT\}$.
- $T$ is the transition function. Moving in a specific direction have a succes chance of $80\%$, and there is a $20\%$ probability of going in the adjacent cells instead.
- $R$ is the reward function: Entering the goal gives $+10$, falling in the trap gives $-10$, and all the other actions gives $0$.






In [ ]:
import gymnasium
from gymnasium import spaces
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

class GridWorld(gymnasium.Env):
  def __init__(self):
    # Define the action and observation spaces
    self.action_space = spaces.Discrete(4) # Up, Down, Left, Right
    self.observation_space = spaces.Discrete(12) # 12 cells

    self.P = { 0: { 0: [[0.9, 0, 0], [0.1, 1, 0]], 1: [[0.8, 4, 0], [0.1, 1, 0], [0.1, 0, 0]], 2: [[0.9, 0, 0], [0.1, 4, 0]], 3: [[0.8, 1, 0], [0.1, 4, 0], [0.1, 0, 0]]},
               1: { 0: [[0.8, 1, 0], [0.1, 0, 0], [0.1, 2, 0]],  1: [[0.8, 1, 0], [0.1, 0, 0], [0.1, 2, 0]], 2: [[0.8, 0, 0], [0.2, 1, 0]], 3: [[0.8, 2, 0], [0.2, 1, 0]]},
               2: { 0: [[0.8, 2, 0], [0.1, 1, 0], [0.1, 3, 0]], 1: [[0.8, 6, 0], [0.1, 1, 0], [0.1, 3, 0]], 2: [[0.8, 1, 0], [0.1, 2, 0], [0.1, 6, 0]], 3: [[0.8, 3, 0], [0.1, 2, 0], [0.1, 6, 0]]},
               3: { 0: [[0.9, 3, 0], [0.1, 2, 0]], 1: [[0.8, 7, -10], [0.1, 2, 0], [0.1, 3, 0]], 2: [[0.8, 2, 0], [0.1, 3, 0], [0.1, 7, -10]], 3: [[0.9, 3, 0], [0.1, 7, -10]]},
               4: { 0: [[0.8, 0, 0], [0.2, 4, 0]], 1: [[0.8, 8, 0], [0.2, 4, 0]], 2: [[0.8, 4, 0], [0.1, 0, 0], [0.1, 8, 0]], 3: [[0.8, 4, 0], [0.1, 0, 0], [0.1, 8, 0]]},
               5: { 0: [[1, 5, 0]], 1: [[1, 5, 0]], 2: [[1, 5, 0]], 3: [[1, 5, 0]]},
               6: { 0: [[0.8, 2, 0], [0.1, 6, 0], [0.1, 7, -10]], 1: [[0.8, 10, 0], [0.1, 6, 0], [0.1, 7, -10]], 2: [[0.8, 6, 0], [0.1, 10, 0], [0.1, 2, 0]], 3: [[0.8, 7, -10], [0.1, 2, 0], [0.1, 10, 0]]},
               7: { 0: [[1, 7, -10]], 1: [[1, 7, -10]], 2: [[1, 7, -10]], 3: [[1, 7, -10]]},
               8: { 0: [[0.8, 4, 0], [0.1, 9, 0], [0.1, 8, 0]], 1: [[0.9, 8, 0], [0.1, 9, 0]], 2: [[0.9, 8, 0], [0.1, 4, 0]], 3: [[0.8, 9, 0], [0.1, 4, 0], [0.1, 8, 0]]},
               9: { 0: [[0.8, 9, 0], [0.1, 8, 0], [0.1, 10, 0]], 1: [[0.8, 9, 0], [0.1, 8, 0], [0.1, 10, 0]], 2: [[0.8, 8, 0], [0.2, 9, 0]], 3: [[0.8, 10, 0], [0.2, 9, 0]]},
               10: { 0: [[0.8, 6, 0], [0.1, 9, 0], [0.1, 11, 10]], 1: [[0.8, 10, 0], [0.1, 9, 0], [0.1, 11, 10]], 2: [[0.8, 9, 0], [0.1, 6, 0], [0.1, 10, 0]], 3: [[0.8, 11, 10], [0.1, 6, 0], [0.1, 10, 0]]},
               11: { 0: [[1, 11, 10]], 1: [[1, 11, 10]], 2: [[1, 11, 10]], 3: [[1, 11, 10]]},
              } 

    # Initialize the state
    self.state = 0

  def step(self, action: int):

    self._transition(action)

    done = False

    reward = 0
    if self.state == 11:
      reward = 10
      done = True
    elif self.state == 7:
      reward = -10
      done = True


    # Return the observation, reward, done flag, and info
    return self.state, reward, done, {}

  def _transition(self, action: int):
    """
    Transition function.
    :param action: Action to take
    """
    r = np.floor(self.state / 3)
    c = self.state % 4
    if action == 0:
      r = max(0, r - 1)
    elif action == 1:
      r = min(2, r + 1)
    elif action == 2:
      c = max(0, c - 1)
    elif action == 3:
      c = min(3, c + 1)
    self.state = r * 4 + c

  def reset(self):
    """
    Reset the environment.
    """
    self.state = 0
    return self.state

  def render(self, render="human"):
    fig, ax = plt.subplots()
    ax.set_xlim(0, 4)
    ax.set_ylim(0, 3)
    ax.set_aspect('equal')


    for i in range(4):
      for j in range(3):
        if j * 4 + i == 11:
          rect = Rectangle((i, j), 1, 1, edgecolor='black', facecolor='green')
          ax.add_patch(rect)
        elif j * 4 + i == 7:
          rect = Rectangle((i, j), 1, 1, edgecolor='black', facecolor='red')
          ax.add_patch(rect)
        elif j * 4 + i == 5:
          rect = Rectangle((i, j), 1, 1, edgecolor='black', facecolor='grey')
          ax.add_patch(rect)
        else:
          rect = Rectangle((i, j), 1, 1, edgecolor='black', facecolor='white')
          ax.add_patch(rect)

    ax.tick_params(axis='both',       # changes apply to both axis
                    which='both',      # both major and minor ticks are affected
                    bottom=False,      # ticks along the bottom edge are off
                    top=False,         # ticks along the top edge are off
                    left=False,
                    right=False,
                    labelbottom=False,
                    labelleft=False) # labels along the bottom edge are off

    plt.show()


In [ ]:
def value_iteration(env, gamma=0.9, theta=0.0001):
  V = np.zeros(env.observation_space.n)
  while True:
    delta = 0
    for s in range(env.observation_space.n):
      v = V[s]
      V[s] = max([sum([p * (r + gamma * V[s_]) for p, s_, r in env.P[s][a]]) for a in env.P[s]])
      delta = max(delta, np.abs(v - V[s]))
      V[7] = -10
      V[11] = 10
    if delta < theta:
      break
  pi = np.zeros(env.observation_space.n)
  for s in range(env.observation_space.n):
    pi[s] = np.argmax([sum([p * (r + gamma * V[s_]) for p, s_, r in env.P[s][a]]) for a in env.P[s]])
  return V, pi

In [ ]:
from IPython.display import display, clear_output

def value_iteration_interact(env, gamma=0.9, theta=0.0001, step=0):
  V = np.zeros(env.observation_space.n)
  for i in range(step):
    delta = 0
    for s in range(env.observation_space.n):
      v = V[s]
      V[s] = max([sum([p * (r + gamma * V[s_]) for p, s_, r in env.P[s][a]]) for a in env.P[s]])
      delta = max(delta, np.abs(v - V[s]))
      V[7] = -10
      V[11] = 10
    if delta < theta:
      break
  pi = np.zeros(env.observation_space.n)
  for s in range(env.observation_space.n):
    pi[s] = np.argmax([sum([p * (r + gamma * V[s_]) for p, s_, r in env.P[s][a]]) for a in env.P[s]])
  return V, pi

def render(labels):
  fig, ax = plt.subplots()
  ax.set_xlim(0, 4)
  ax.set_ylim(0, 3)
  ax.set_aspect('equal')

  for i in range(4):
    for j in range(3):
      if j * 4 + i == 11:
        rect = Rectangle((i, j), 1, 1, edgecolor='black', facecolor='green')
        ax.add_patch(rect)
      elif j * 4 + i == 7:
        rect = Rectangle((i, j), 1, 1, edgecolor='black', facecolor='red')
        ax.add_patch(rect)
      elif j * 4 + i == 5:
        rect = Rectangle((i, j), 1, 1, edgecolor='black', facecolor='grey')
        ax.add_patch(rect)
      else:
        rect = Rectangle((i, j), 1, 1, edgecolor='black', facecolor='white')
        ax.add_patch(rect)
      ax.text(i + 0.5, j + 0.5, int(labels[j * 4 + i]), ha='center', va='center')

  ax.tick_params(axis='both',        # changes apply to both axis
                  which='both',      # both major and minor ticks are affected
                  bottom=False,      # ticks along the bottom edge are off
                  top=False,         # ticks along the top edge are off
                  left=False,
                  right=False,
                  labelbottom=False,
                  labelleft=False)   # labels along the bottom edge are off
  display(fig)
  plt.close(fig)

def update(gamma=0.9, step=0):
  V, pi = value_iteration_interact(env, gamma, step=step)
  render(V)

In [ ]:
from ipywidgets import *

env = GridWorld()
interact(update, gamma = (0.5,0.9,0.1), step = (0, 50, 1));

In [ ]:
import plotly.graph_objects as go
import numpy as np

def value_iteration_interact(env, gamma=0.9, theta=0.0001, step=0):
  V = np.zeros(env.observation_space.n)
  V[7] = -10
  V[11] = 10
  for i in range(step):
    delta = 0
    for s in range(env.observation_space.n):
      v = V[s]
      V[s] = max([sum([p * (r + gamma * V[s_]) for p, s_, r in env.P[s][a]]) for a in env.P[s]])
      delta = max(delta, np.abs(v - V[s]))
      V[7] = -10
      V[11] = 10
    if delta < theta:
      break
  pi = np.zeros(env.observation_space.n)
  for s in range(env.observation_space.n):
    pi[s] = np.argmax([sum([p * (r + gamma * V[s_]) for p, s_, r in env.P[s][a]]) for a in env.P[s]])
  return V, pi


env = GridWorld()
# Create figure
fig = go.Figure()

# Add traces, one for each slider step
for step in range(0, 15):
    V, _ = value_iteration_interact(env, 0.9, step=step) 
    x, y, text = [], [], []
    for i in range(4):
      for j in range(3):
        if j * 4 + i == 11:
          fig.add_shape(type="rect", x0=i, y0=j, x1=i + 1, y1=j + 1, line=dict(color="black"), fillcolor="green", layer="below")
        elif j * 4 + i == 7:
          fig.add_shape(type="rect", x0=i, y0=j, x1=i + 1, y1=j + 1, line=dict(color="black"), fillcolor="red", layer="below")
        elif j * 4 + i == 5:
          fig.add_shape(type="rect", x0=i, y0=j, x1=i + 1, y1=j + 1, line=dict(color="black"), fillcolor="gray", layer="below")
        else:
          fig.add_shape(type="rect", x0=i, y0=j, x1=i + 1, y1=j + 1, line=dict(color="black"), fillcolor="white", layer="below")
        x.append(i + 0.5)
        y.append(j + 0.5)
        text.append(int(V[j * 4 + i]))
    fig.add_trace(go.Scatter(
      x=x,
      y=y,
      text=text,
      mode="text",
      textfont=dict(
          color="black",
          size=18,
          family="Arail",
      ),
      visible=False
    )
)
fig.data[0].visible = True
# Create and add slider
steps = []
for i in range(len(fig.data)):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data)}],  # layout attribute
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    active=0,
    pad={"t": 50},
    steps=steps
)]

fig.update_xaxes(
    showticklabels=False,
    showgrid=False,
    zeroline=False,
    automargin=True
)

fig.update_yaxes(
    showticklabels=False,
    showgrid=False,
    zeroline=False,
    scaleanchor="x",
    scaleratio=1,
    automargin=True
)

fig.update_layout(
    autosize=False,
    sliders=sliders
)

fig.show()
